In [ ]:
import yaml
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from SimBMVtool import SimBMVtool as sim

path_config = './config_simu.yaml'

In [ ]:
accmodel = sim.BaseSimBMVCreator()
accmodel.load_true_background_irfs(path_config) 

# If you want to get the gammapy BackgroundIRF without knowing the name of the variables
true = accmodel.get_background_irf(type='true',downsampled=False) 
true.plot_at_energy(["1 TeV"],figsize=(3,3))
plt.show()

# You can also plot directly from the class object.
if accmodel.true_collection: accmodel.bkg_true_irf[0].peek()
else: accmodel.bkg_true_irf.peek()

In [ ]:
do_simulation = True
if do_simulation: accmodel.do_simulation()
else: accmodel.load_observation_collection()

In [ ]:
accmodel.obs_collection[1].bkg.peek()
accmodel.obs_collection[1].events.peek()